# Normalized Difference Water Index (NDWI) 

Articulo de referencia
 
https://www.sciencedirect.com/science/article/abs/pii/S0034425796000673

El Índice de Agua de Diferencia Normalizada (NDWI) fue desarrollado por Gao (1996) como un índice del contenido de agua de la vegetación. El índice es sensible a los cambios en el contenido líquido de los doseles vegetales. Esto significa que el índice se puede utilizar, por ejemplo, para detectar vegetación que experimenta condiciones de sequía o diferenciar los niveles de riego de los cultivos. En áreas secas, los cultivos que se irrigan se pueden diferenciar de la vegetación natural. También se denomina a veces Índice de Humedad de Diferencia Normalizada (NDMI).



**How it Works:**
 
**Healthy Vegetation:**
- High NIR reflectance (due to leaf structure).
- Low SWIR reflectance (water strongly absorbs SWIR).
- Result: Positive NDWI values indicate high water content.
 
**Stressed or Dry Vegetation:**

- NIR reflectance decreases slightly, but SWIR reflectance increases significantly (less water absorption).
- Result: NDWI values decrease, approaching zero or negative, indicating low water content.


**Interpretation:**
- High NDWI (Near 1) – Vegetation with high water content (healthy).
- Moderate NDWI (0 to 0.5) – Vegetation with moderate water content.
- Low NDWI (Negative values) – Stressed or dry vegetation


**Applications:**
- Agriculture – Monitor crop water stress and optimize irrigation.
- Forestry – Assess drought conditions in forests and detect forest fires.
- Climate Studies – Track large-scale vegetation response to climate change.
- Disaster Management – Identify areas of vegetation at risk of drought or fire.

La formula del NDWI es la siguiente: 

<div style="text-align: center;">
    <img src="Images/NDWI formula.png" alt="Descripción de la imagen" width="500">
</div>
 

**BANDAS NECESARIAS:** 
 
En este caso utilizaremos imagenes de Sentinel 2. Por ende, hay que encontrar cuales son las bandas en sentinel 2 que corresponden a estas longitudes de onda. 

En el caso de Sentinel 2 la banda NIR (Near Infrared) es la banda **8** con una longitud de onda de Central Wavelength = 842nm.  Por otra parte, la banda de SWIR (short-wave infrared) es mejor el uso de la banda 11 que tiene un centro de longitud de onda  = 1610nm. 

**NOTA:** El rnago de SWIR adecuados aparentmenete son entre 1,400 a 1,900 nm. 



**IMPORTACION DE LIBRERIAS**

In [4]:
import os 
import ee 
import geemap
import pandas as pd 

**AUTENTIFICACION**

In [3]:
# 1) Obtención de la dirección de trabajo 
direction = os.getcwd()

# Remove the last part of the path
direction = os.path.dirname(direction)

# Dirección de la llave 
service_account = direction + '/conf/local/gcp-for-data-science-397913-4fd843feede1.json'

# Autentificación 
credentials = ee.ServiceAccountCredentials(email=None, key_file=service_account)
ee.Initialize(credentials)

## 1) Lectura y procesamiento de imagenes  
Las imagenes satelitales se almacenaron en un bucket de GCP, por lo que serán leídas de ahi mismo.

API Reference:
 
  https://developers.google.com/earth-engine/apidocs/ee-image-loadgeotiff

### 1.1) Cálculo y exportación de imagenes con NDWI

In [ ]:
### AQUI VOY !! , tengo que modificar el ciclo para que calcule el ndwi. 


In [3]:
# 1) Lectura del nombre de las imagenes en el bucket 
images_names = pd.read_csv(filepath_or_buffer=direction + "/data/02_intermediate/images_names.csv")

# diccionario de imagenes 
images_dic = {}

# 2) Inicio del ciclo 
for i in images_names["Images names"]:
    
    # 1) Descarga de imagenes 
    # Path to the image in the GCS bucket
    gcs_path = f'gs://earth_engine_selene/sentinel2/{i}.tif'
    
    # Load the image from the GCS bucket
    image = ee.Image.loadGeoTIFF(gcs_path)
     
    # 2) Calculate NDVI (if bands are organized)
    image = image.normalizedDifference(['B8', 'B4']).rename('NDVI')  # Renombre de la banda resultante (capa) 
    print(f"NDVI Calculated for {i}")
    
    # 3) Generaación de URL 
    rgb_bands = ["NDVI"]

    ndvi_vis = {
                "min": 0,
                "max": 1,
                "palette": ["red", "yellow", "green"], 
    "bands":rgb_bands}

    url = image.getThumbURL(ndvi_vis)
    print(f"Thumbnail URL for {i}: {url}")
        
    # 4) Descarga de los datos 
    # Download the image and save it locally
    output_file = direction + f"/data/07_model_output/{i}.png"
    
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print(f"Image successfully downloaded as '{output_file}'")
    else:
        print(f"Error: Unable to download the image. HTTP status code {response.status_code}")
    
    images_dic[i]  = image

NDVI Calculated for image_2023-1
Thumbnail URL for image_2023-1: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/5bdbb88cdd7920d17c235b1703cbecc8-a9d44e2cb8422c78cb799a06b742ff39:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/image_2023-1.png'
NDVI Calculated for image_2023-3
Thumbnail URL for image_2023-3: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/1b059d6e797423d39b16590e4c74a088-73938f4a728c81ce384c17386b5cd5e0:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/image_2023-3.png'
NDVI Calculated for image_2023-5
Thumbnail URL for image_2023-5: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/58e58f7585a29879b275da0a9dfb2215-d5f867b8bc8c55cc752f5c361668386d:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites

In [ ]:
### SUGERENCIA DE CHAT GPT: 

import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Load a Sentinel-2 image (TOA or SR)
image = ee.Image('COPERNICUS/S2_SR/20240101T180011_20240101T180013_T10SEG')

# Select NIR (B8) and SWIR (B11) bands
nir = image.select('B8')  # NIR
swir = image.select('B11')  # SWIR

# Calculate NDWI using normalizedDifference
ndwi = image.normalizedDifference(['B8', 'B11']).rename('NDWI')

# Define visualization parameters
vis_params = {
    'min': -0.5,
    'max': 1,
    'palette': ['brown', 'yellow', 'green']
}

# Visualize the NDWI
Map = geemap.Map()
Map.centerObject(image, 9)
Map.addLayer(ndwi, vis_params, 'NDWI (Gao 1996)')
Map.addLayer(image.select(['B4', 'B3', 'B2']), {'min': 0, 'max': 3000}, 'True Color')
Map
